# Project Final Report: Predicting subscription status
Group number: 25

Authors: Dylan Lucas, Julie Wang, Yingnian Le

Output: html_document

In [5]:
# load packages
library(tidyverse)
library(tidymodels)
library(lubridate)
options(repr.matrix.max.rows = 6)

## Introduction:

A research group at UBC, led by Frank Wood, has set up a Minecraft server where they're aiming to create an advanced artificial intelligence that can not only respond to speech, but also behave in a virtual space like Minecraft, just like humans. (Pacific Laboratory of Artificial Intelligence, 2025) To do this, they're collecting data on how humans navigate the Minecraft world.

They need help targeting their use of resources to handle the number of players that sign up. There are three questions of interest:
- Question 1: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?
- Question 2: We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.
- Question 3: We are interested in demand forecasting, namely, what time windows are most likely to have large number of simultaneous players. This is because we need to ensure that the number of licenses on hand is sufficiently large to accommodate all parallel players with high probability. 

**In this report, we are trying to answer Question 1.** Specifically, can player experience level, playtime, gender, and/or age predict whether a player subscribes to the newsletter? (**NOTE: discuss what specific question we want to do. we don't have to use all the variables, we can pick and choose. we could also explore predictor variable selection as presented in chapter 6.8 of the textbook?**)

We've been given data on the players that have signed up (players.csv), as well as their corresponding play sessions on the server (sessions.csv). As we are looking at player characteristics and behaviours, we will only be looking at players.csv.

players.csv: (table from Yingnian)
| Variable     | Type        | Description                                                                 |
|---------------|-------------|------------------------------------------------------------------------------|
| experience    | Categorical | Player experience level (e.g., “Amateur”, “Pro”, “Veteran”)                 |
| subscribe     | Logical     | Whether the player subscribed to the game-related newsletter (TRUE/FALSE)   |
| hashedEmail   | String      | Unique hashed identifier for each player                                    |
| played_hours  | Numeric     | Total number of hours the player spent in the game                          |
| name          | String      | Player’s name                                                               |
| gender        | Categorical | Player gender                                                               |
| Age           | Numeric     | Player age                                                                  |

(**NOTE: in the table, experience and gender are written as categorical variables, but in the unwrangled dataset it is a character type variable. discuss later?**)

There are 196 observations and 7 variables. 
The average play time is 5.85 hours, and the average age is 21.14 years. (Summary statistics are only given for numerical values.) (**NOTE: we can maybe find the most common experience/gender for the categorical values?**)

Some potential issues in the data are: 
- Possible missing values in played_hours or Age. (There are users who signed up but did not log any hours, and some users did not provide an age.)
- All variables are self-reported (except for played_hours). People can easily lie about their information, and this is especially common for age. 
- When hashing emails, the password that decrypts the emails should be well-protected to avoid issues regarding data leaks.

The data was likely collected via a sign up form, where users make an profile and provide the information presented in the dataset. Then, any time they play, the hours are logged and linked to their profile. 

## Methods & Results:

**describe the methods you used to perform your analysis from beginning to end that narrates the analysis code**

your report should include code which:
- loads data 
- wrangles and cleans the data to the format necessary for the planned analysis
- performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
- creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
- performs the data analysis
- creates a visualization of the analysis 
- note: all figures should have a figure number and a legend


In [24]:
# loading data
players_data <- read_csv("data/players.csv")
players_data

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,57
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


In [25]:
# wrangling data, only keeping variables relevant for answering the question
# changing subscribe to a factor type variable for classification later

players <- players_data |>
    mutate(experience = as_factor(experience), gender = as_factor(gender), subscribe = as_factor(subscribe)) |>
    mutate(subscribe = fct_recode(subscribe, "Yes" = "TRUE", "No" = "FALSE")) |>
    select(subscribe, experience, played_hours, gender, Age)
players

subscribe,experience,played_hours,gender,Age
<fct>,<fct>,<dbl>,<fct>,<dbl>
Yes,Pro,30.3,Male,9
Yes,Veteran,3.8,Male,17
No,Veteran,0.0,Male,17
⋮,⋮,⋮,⋮,⋮
No,Amateur,0.0,Prefer not to say,57
No,Amateur,2.3,Male,17
Yes,Pro,0.2,Other,NA


In [26]:
# summary of data set for exploratory data analysis

In [14]:
# visualization of data set for exploratory data analysis

In [28]:
# classification data analysis (code from Yingnian)

set.seed(123)
players_split <- initial_split(players, prop = 0.7, strata = subscribe)
players_train <- training(players_split)
players_test <- testing(players_split)

player_recipe <- recipe(subscribe ~ experience + played_hours + Age + gender, data = players_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_mode("classification") |>
  set_engine("kknn")

knn_workflow <- workflow() |>
  add_recipe(player_recipe) |>
  add_model(knn_spec)

In [16]:
# visualization of classification data analysis

## Discussion:
**summarize what you found**
- discuss whether this is what you expected to find
- discuss what impact could such findings have
- discuss what future questions could this lead to


## References
You may include references if necessary, as long as they all have a consistent citation style.

Pacific Laboratory of Artificial Intelligence. (2025). PLAICraft. https://plaicraft.ai/ 